In [1]:
from pathlib import Path

In [2]:
import cv2

ImportError: /opt/ros/kinetic/lib/python2.7/dist-packages/cv2.so: undefined symbol: PyCObject_Type

In [16]:
from skimage.transform import resize

In [3]:
from skimage.io import imread, imsave

In [28]:
USE_CV = False

def load_img(path):
    if USE_CV:
        return cv2.imread(path)
    else:
        return imread(path)

def save_img(path, img):
    if USE_CV:
        return cv2.imwrite(path, img)
    else:
        return imsave(path, img)
    
def resize_img(img, size):
    if USE_CV:
        return cv2.resize(img, size)
    else:
        return resize(
            img,
            size,
            mode="reflect"
            #anti_aliasing=True
        )

In [5]:
size = (188, 620)

In [6]:
roots = Path('.').iterdir()
roots=list(roots)
roots = [root for root in roots if root.is_dir() and '2011_' in root.name]
roots

[PosixPath('2011_09_26')]

In [29]:
timgs0, timgs1, timgs2, paras = [], [], [], []
for root in roots:
    #para = get_campara(root)
    folders = root.glob("*_sync")
    for folder in folders:
        img02 = folder /'image_02'/'data'
        img03 = folder /'image_03'/'data'
        img02r = folder /'image_02_{}_{}'.format(*size)/'data'
        img03r = folder /'image_03_{}_{}'.format(*size)/'data'
        
        img02r.mkdir(parents=True, exist_ok=True)
        img03r.mkdir(parents=True, exist_ok=True)
        for which, which_r in ((img02, img02r), (img03, img03r)):
            imgs = sorted([img for img in which.iterdir()])
            
            for img in imgs:
                resized = resize( load_img(str(img)), size)
                save_img(which_r/img.name, resized)

/home/aurora/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/aurora/.local/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [39]:
import pandas as pd
import numpy as np

In [40]:
from collections import namedtuple

In [41]:
CAMPARA = namedtuple('campara',['fx','fy','cx','cy'])
def get_campara(date):
    cam = date/'calib_cam_to_cam.txt'
    camret = {}
    with cam.open('r') as f:
        for l in f:
            kv = l.rstrip().split(':')
            k, v = kv[0], kv[1]
            v= v.split(' ')[1:]
            camret[k] = v
    cam2para = np.array(tuple(float(e) for e in camret['P_rect_02'])).reshape(3,4)
    return CAMPARA(fx = cam2para[0,0], fy = cam2para[1,1], cx = cam2para[0,2], cy = cam2para[1,2])

In [43]:
def gene_tri(imgs):
    return imgs[0:-2], imgs[1:-1], imgs[2:]

In [44]:
timgs0, timgs1, timgs2, paras = [], [], [], []
for root in roots:
    para = get_campara(root)
    folders = root.glob("*_sync")
    for folder in folders:
        img02r = folder /'image_02_{}_{}'.format(*size)/'data'
        img03r = folder /'image_03_{}_{}'.format(*size)/'data'
        
        for which in (img02r, img03r):
            imgs = sorted([str(img) for img in which.iterdir()])
            imgs0, imgs1, imgs2 = gene_tri(imgs)
            timgs0 += imgs0
            timgs1 += imgs1
            timgs2 += imgs2
            paras += [para]*len(imgs0)
paras = tuple(zip(*paras))

In [45]:
train = pd.DataFrame(dict(t0=timgs0, t1=timgs1,t2=timgs2,
                          fx=paras[0],fy=paras[1],
                          cx=paras[2],cy=paras[3])
                    )
train.head()

val = train.iloc[int(len(train)*0.9):]
train = train.iloc[:int(len(train)*0.9)]

val.to_csv('validation_{}_{}.csv'.format(*size))
train.to_csv('training{}_{}.csv'.format(*size));